In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from obspy.taup import TauPyModel
from obspy.geodetics import degrees2kilometers, kilometer2degrees, gps2dist_azimuth
from mudpy import forward

In [2]:
path = '/Users/sydneydybing/SW4/strain/experiments/realistic_eq_sizes/'
stas = pd.read_csv(path + 'stas_w_p_s_traveltimes.csv')
hypo_lat = 35.7793
hypo_lon = -117.6204
hypo_depth = 7.5
mcmc_results = pd.read_csv(path + 'filter_mcmc_arrays/mcmc_results_summary_with_quality.csv')
good_results = mcmc_results[mcmc_results['two_line_fit_qual'] == 'g']
bad_results = mcmc_results[mcmc_results['two_line_fit_qual'] == 'b']

In [3]:
mcmc_results

,eq_id,mag,sta,hypdist,xhigh,m1,m2,xinter,sigma,trans_time,p_arrival,time_to_transition,two_line_fit_qual
0,M5,5,E18,111,25,1.188355,0.050015,19.382357,0.069985,19.40,18.567198,0.815159,g
1,M5,5,E36,97,20,7.781424,0.141056,16.533789,0.129427,16.61,16.298455,0.235334,g
2,M5,5,E54,83,19,5.764275,0.128457,14.249171,0.091895,14.31,14.014324,0.234847,g
3,M5,5,E72,68,15,1.680614,0.134060,12.122745,0.070798,12.20,11.684379,0.438366,g
4,M5,5,E90,54,12,1.408662,0.268101,9.656373,0.083540,9.70,9.356268,0.300105,g
5,M5,5,E108,40,10,12.597585,0.171319,7.215629,0.099513,7.30,7.031333,0.184296,g
6,M5,5,E126,25,7,16.106269,0.168572,4.851396,0.087575,4.90,4.717005,0.134390,g
7,M5,5,E144,11,4,2.377545,0.240919,3.954037,0.131083,2.50,2.483539,1.470498,g
8,M6,6,E18,111,25,1.327085,0.119745,19.258559,0.060562,19.30,18.567198,0.691361,g
9,M6,6,E36,97,20,3.646150,0.197210,16.652343,0.078309,16.71,16.298455,0.353887,g


In [4]:
files_list = os.listdir(path)
# print(files_list)

rupts_list = []
for file in files_list:
    if file[-4:] == 'rupt':
        rupts_list.append(file)

In [6]:
stas_sub = ['E144', 'E126', 'E108', 'E90', 'E72', 'E54', 'E36', 'E18']
stas_sub = ['E144', 'E108', 'E72', 'E36']
Mws = [5, 6, 7, 8]
colors = ['blue', 'green', 'orange', 'red']
letters = ['(a)', '(b)', '(c)', '(d)']
fig = plt.figure(figsize = (8,6), dpi = 400, facecolor = 'white')

for idx in range(len(stas_sub)):

    sta = stas_sub[idx]

    i = np.where(stas.name.values == sta)[0]
    hypdist = stas.hypo_dist_km.values[i][0]
    theo_p_arr = stas.p_arrival.values[i][0]
    theo_s_arr = stas.s_arrival.values[i][0]

    ax = fig.add_subplot(2, 2, idx+1)
    # ax.set_title(sta)
    # ax.axvline(theo_s_arr, color = 'skyblue', linestyle = '--', label = 'S-wave arrival')

    for idx2 in range(len(Mws)):
        
        Mw = Mws[idx2]
        color = colors[idx2]

        model_path = path + 'talapas_outputs/M' + str(Mw) + '_results/'
        
        strain_path = model_path + sta + 's.txt'
        disp_path = model_path + sta + 'd.txt'

        time_strain, xx_strain, xy_strain, yy_strain, rms_strain, peak_strain, norm_peak_strain = process_strain(strain_path, theo_p_arr)
        time_gnss, x_gnss, y_gnss, rms_gnss, peak_gnss, norm_peak_gnss = process_disp(disp_path, theo_p_arr)

        ax.plot(time_strain, norm_peak_strain, color = color, label = 'M' + str(Mw))
        
    ax.set_yscale('log')
    ax.set_ylabel('Peak strain')
    ax.set_xlabel('Time (s)')
    ax.set_xlim(theo_p_arr-0.5, theo_s_arr+5)
    ax.text(x = 0.75, y = 0.1, s = sta, fontsize = 15, transform = ax.transAxes) 
    ax.text(x = -0.32, y = 0.9, s = letters[idx], fontsize = 20, transform = ax.transAxes)
    ax.axvline(theo_p_arr, color = 'gray', linestyle = '--', label = 'P-wave arrival')
    

    if idx == int(len(stas_sub)-1):
        ax.legend(bbox_to_anchor = [0.7, -0.28], ncol = 5)

plt.subplots_adjust(hspace = 0.25, wspace = 0.35, bottom = 0.18)

# plt.show();

# plt.savefig('/Users/sydneydybing/StrainProject/2024/Manuscript/Figures/renamed_figs/Figure_10.jpg', format = 'JPG')
plt.savefig('/Users/sydneydybing/dissertation/defense_figs/strain/Figure_10.png', format = 'PNG')
plt.close();

In [ ]:
# Steeper slope in log-space for higher magnitudes
# Later transition time in log-space for higher magnitudes
# Strong determinism shows up in these models, but not in real life

In [5]:
### Process strain data ###

def process_strain(strain_data, theoretical_p_arrival):
    
    strain = pd.read_csv(strain_path, sep = '\s+', skiprows = 16, names = ['time', 'xx', 'yy', 'zz', 'xy', 'xz', 'yz'])

    # Calculate horizontal RMS strain
    time_strain = strain.time.values
    xx_strain = strain.xx.values
    xy_strain = strain.xy.values
    yy_strain = strain.yy.values
    
    rms_strain = np.sqrt((xx_strain**2 + xy_strain**2 + yy_strain**2)/3)

    # Calculate peak strain
    strain_p_i = np.abs(time_strain - theo_p_arr).argmin() # Get the index where the P-wave arrives
    mod_rms_strain = rms_strain.copy()
    p_arr_strain = rms_strain[strain_p_i]
    mod_rms_strain[:strain_p_i] = p_arr_strain # Strain before the P wave is set to be the same as the strain AT the P wave
    peak_strain = mod_rms_strain.copy()
    for k in range(0,len(mod_rms_strain)):
        if k == 0:
            strain = mod_rms_strain[0]
            max_strain = strain
        else:
            # Grab progressively longer windows and save the biggest strain
            strain = mod_rms_strain[:k+1] # Has to be k+1 because slicing doesn't include last one
            max_strain = max(strain)
        # Put peak strain back into the output stream
        peak_strain[k] = max_strain 
    
    # Normalize to 10**-9
    norm_peak_strain = np.copy(peak_strain)
    norm_peak_strain_level = peak_strain[0]
    norm_peak_strain = norm_peak_strain - norm_peak_strain_level + 1*10**-9
    
    # Filtering stuff
    
    max_time = max(time_strain)
    npts = len(time_strain)
    sampling_rate = npts / max_time
    # print(sampling_rate) 
    
    # Calculate high-pass filtered horizontal RMS strain
    corner_period = 1000 # seconds
    corner_freq = 1/corner_period

    filter_xx_strain = forward.highpass(xx_strain, fcorner = corner_freq, fsample = sampling_rate, order = 4)
    filter_xy_strain = forward.highpass(xy_strain, fcorner = corner_freq, fsample = sampling_rate, order = 4)
    filter_yy_strain = forward.highpass(yy_strain, fcorner = corner_freq, fsample = sampling_rate, order = 4)

    # Detrend
    p_arr_samp = int(sampling_rate * theo_p_arr)
    filter_xx_strain = filter_xx_strain - np.mean(filter_xx_strain[p_arr_samp-2:p_arr_samp+2])
    filter_xy_strain = filter_xy_strain - np.mean(filter_xy_strain[p_arr_samp-2:p_arr_samp+2])
    filter_yy_strain = filter_yy_strain - np.mean(filter_yy_strain[p_arr_samp-2:p_arr_samp+2])
    
    filter_rms_strain = np.sqrt(((filter_xx_strain)**2 + (filter_xy_strain)**2 + (filter_yy_strain)**2)/3)  
    
    # Calculate filtered peak strain
    strain_p_i = np.abs(time_strain - theo_p_arr).argmin() # Get the index where the P-wave arrives
    filter_mod_rms_strain = filter_rms_strain.copy()
    filter_p_arr_strain = filter_rms_strain[strain_p_i]
    filter_mod_rms_strain[:strain_p_i] = filter_p_arr_strain # Strain before the P wave is set to be the same as the strain AT the P wave
    filter_peak_strain = filter_mod_rms_strain.copy()
    for k in range(0,len(filter_mod_rms_strain)):
        if k == 0:
            strain = filter_mod_rms_strain[0]
            max_strain = strain
        else:
            # Grab progressively longer windows and save the biggest strain
            strain = filter_mod_rms_strain[:k+1] # Has to be k+1 because slicing doesn't include last one
            max_strain = max(strain)
        # Put peak strain back into the output stream
        filter_peak_strain[k] = max_strain 
    
    # Normalize to 10**-9
    norm_peak_strain = np.copy(filter_peak_strain)
    norm_peak_strain_level = filter_peak_strain[0]
    norm_peak_strain = norm_peak_strain - norm_peak_strain_level + 1*10**-9
    
    return time_strain, filter_xx_strain, filter_xy_strain, filter_yy_strain, filter_rms_strain, filter_peak_strain, norm_peak_strain
    # return time_strain, xx_strain, xy_strain, yy_strain, rms_strain, peak_strain, norm_peak_strain

### Process GNSS data to match the strain processing ###

def process_disp(disp_data, theoretical_p_arrival):
    
    disp = pd.read_csv(disp_path, sep = '\s+', skiprows = 13, names = ['time', 'x', 'y', 'z'])

    # Calculate horizontal RMS GNSS  
    time_gnss = disp.time.values
    x_gnss = disp.x.values
    y_gnss = disp.y.values
    rms_gnss = np.sqrt((x_gnss**2 + y_gnss**2)/2)

    # Calculate peak GNSS
    gnss_p_i = np.abs(time_gnss - theo_p_arr).argmin() # Get the index where the P-wave arrives
    mod_rms_gnss = rms_gnss.copy()
    p_arr_gnss = rms_gnss[gnss_p_i]
    mod_rms_gnss[:gnss_p_i] = p_arr_gnss # Strain before the P wave is set to be the same as the strain AT the P wave
    peak_gnss = mod_rms_gnss.copy()
    for k in range(0,len(mod_rms_gnss)):
        if k == 0:
            gnss = mod_rms_gnss[0]
            max_gnss = gnss
        else:
            # Grab progressively longer windows and save the biggest strain
            gnss = mod_rms_gnss[:k+1] # Has to be k+1 because slicing doesn't include last one
            max_gnss = max(gnss)
        # Put peak strain back into the output stream
        peak_gnss[k] = max_gnss 
        
    # Normalize to 10**-6
    norm_peak_gnss = np.copy(peak_gnss)
    norm_peak_gnss_level = peak_gnss[0]
    norm_peak_gnss = norm_peak_gnss - norm_peak_gnss_level + 1*10**-6
    
    return time_gnss, x_gnss, y_gnss, rms_gnss, peak_gnss, norm_peak_gnss